In [1]:
import pandas as pd
import json

file_path = '/Users/jamestreadwell/CompAnalysis/BigDataFinalProject/signalmedia-1m.jsonl'

left_sources = {"cnn", "the new york times", "vox.com", "msnbc", "buzzfeed", "the new yorker: news"}
right_sources = {"fox news", "breitbart.com", "theblaze.com - stories", "cbn.com - cbn news", "daily caller", "daily mail", "newsmax.com", "new york post", "the american conservative \u00bb daniel larison", "infowars", "redstate", "national review"}

left_rows = []
right_rows = []

with open(file_path, 'r') as file:
    for line in file:
        entry = json.loads(line)
        if entry.get('source', '').lower().strip() in left_sources:
            left_rows.append(entry)
        if entry.get('source', '').lower().strip() in right_sources:
            right_rows.append(entry)
        if len(left_rows) + len(right_rows) >= 3000:
            break

left_data = pd.DataFrame(left_rows)
right_data = pd.DataFrame(right_rows)

combined_data = pd.concat([left_data, right_data], ignore_index=True)

In [10]:
##print(combined_data[['content', 'source']].head())
print(combined_data.head())

                                     id  \
0  693ae030-4239-4291-b248-dbf7c1696ff2   
1  e7157bbf-6ad4-477d-a4e4-14a0bb9ad9f5   
2  0ef6b514-620d-4e8a-9405-e390e16c6cd7   
3  96b993b2-320f-457a-bab2-6bc214c05013   
4  c3a3224d-bae5-40e3-80f6-0ca74083beaf   

                                             content  \
0  Georgia is set to execute its only woman on de...   
1  Story highlights Family posts, "We are devasta...   
2  Obama dropped in on "The Late Show with Stephe...   
3  Story highlights Estonia, a neighbor of Russia...   
4  Story highlights Swiss prosecutors open crimin...   

                                               title media-type    source  \
0  Georgia Plans To Execute Its Only Woman On Dea...       News  BuzzFeed   
1               'Senseless loss of a beautiful life'       News       CNN   
2  Michelle Obama gives Stephen Colbert advice fo...       News       CNN   
3                U.S. stealth fighters visit Estonia       News       CNN   
4      FIFA boss Bl

In [12]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

combined_text = combined_data['content'].tolist()
test_text = combined_data.iloc[0]['content']

print(test_text)


Georgia is set to execute its only woman on death row Tuesday evening. If all goes according to the state’s plan, Kelly Gissendaner will be the first woman the state has executed in 70 years. She was sentenced to death in 1998 for plotting the death of her husband, Douglas Gissendaner, with her boyfriend. 
 
Gissendaner drove her boyfriend, Gregory Owen, to her house, gave him the murder weapons — a nightstick and a hunting knife — and then went to a nightclub with friends, according to court documents. Owen ambushed Douglas and forced him to drive his car to a remote location that Gissendaner chose beforehand. 
 
Once they arrived, Owen beat Douglas and stabbed him. Owen took Douglas’s wedding ring and watch to make it look like a robbery. Gissendaner later arrived at the scene and helped her boyfriend burn her husband’s body and car. Law enforcement was unable to find Douglas’s body for weeks. 
   
Owen accepted a plea deal and testified against Gissendaner. In exchange, he received 

In [14]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jamestreadwell/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
test_tokenized = word_tokenize(test_text)

print(test_tokenized)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/jamestreadwell/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/share/nltk_data'
    - '/Library/Frameworks/Python.framework/Versions/3.12/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
